# Named Entity Recognition (NER) on Financial-News

## Import

In [13]:
from nltk.tokenize import word_tokenize
import os
from nltk import *
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tag.stanford import StanfordNERTagger


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stefa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Preprocessing dataset

In [59]:
df = pd.read_csv('all-data.csv', encoding="latin-1",
                 names=['sentiment', 'headline'])
df = df["headline"]
df.head()

0    According to Gran , the company has no plans t...
1    Technopolis plans to develop in stages an area...
2    The international electronic industry company ...
3    With the new production plant the company woul...
4    According to the company 's updated strategy f...
Name: headline, dtype: object

In [8]:
print("original shape: ", df.shape)
df = df.drop_duplicates()
print("after drop duplicates shape: ", df.shape)
dd_dn = df.dropna()
print("after drop null shape: ", df.shape)

original shape:  (4846,)
after drop duplicates shape:  (4838,)
after drop null shape:  (4838,)


In [9]:
df.isnull().sum()

0

In [10]:
print("news headline example: ", df[0])

news headline example:  According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .


## Stanford NER

In [12]:
java_path = "C:/Program Files/Java/jre-1.8/bin/java.exe"
os.environ['JAVAHOME'] = java_path

model = "C:/Users/stefa/Desktop/financial-news-analysis/stanford-ner/classifiers/english.muc.7class.distsim.crf.ser.gz"
jar = "C:/Users/stefa/Desktop/financial-news-analysis/stanford-ner/stanford-ner-4.2.0.jar"

ner_tagger = StanfordNERTagger(model, jar, encoding="utf-8")

testo = "According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing"

words = word_tokenize(testo)
classified_words = ner_tagger.tag(words)
classified_words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stefa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('According', 'O'),
 ('to', 'O'),
 ('Gran', 'ORGANIZATION'),
 (',', 'O'),
 ('the', 'O'),
 ('company', 'O'),
 ('has', 'O'),
 ('no', 'O'),
 ('plans', 'O'),
 ('to', 'O'),
 ('move', 'O'),
 ('all', 'O'),
 ('production', 'O'),
 ('to', 'O'),
 ('Russia', 'LOCATION'),
 (',', 'O'),
 ('although', 'O'),
 ('that', 'O'),
 ('is', 'O'),
 ('where', 'O'),
 ('the', 'O'),
 ('company', 'O'),
 ('is', 'O'),
 ('growing', 'O'),
 ('.', 'O')]

In [52]:
df.head()

,headline,entity,tag
0,"According to Gran , the company has no plans t...",Gran,ORGANIZATION
